In [24]:
import pandas as pd
import os


In [25]:
diretorio_atual = os.getcwd()
diretorio_dados = os.path.join(diretorio_atual, 'dados')

In [41]:
caminho_dados = os.path.join(diretorio_atual, 'dados')

In [52]:
# ... (imports e definições de caminho continuam iguais)

for arquivo in os.listdir(caminho_dados):
    caminho_arquivo = os.path.join(caminho_dados, arquivo)

    # Lógica para carregar, limpar espaços e adicionar prefixo
    if 'Preventiva' in arquivo:
        df_preventiva = pd.read_excel(caminho_arquivo)
        df_preventiva.columns = (df_preventiva.columns.str.strip().str.upper()) # Limpa espaços antes
        df_preventiva = df_preventiva.add_prefix('PREVENTIVA_')   # Adiciona prefixo
        
    elif 'CARRETA' in arquivo:
        df_carreta = pd.read_excel(caminho_arquivo)
        df_carreta.columns = df_carreta.columns.str.strip()
        df_carreta = df_carreta.add_prefix('CARRETA_')
        
    elif 'magazine' in arquivo:
        df_esl = pd.read_excel(caminho_arquivo)
        df_esl.columns = df_esl.columns.str.strip().str.upper()
        df_esl = df_esl.add_prefix('ESL_')
        
    elif 'Mobile' in arquivo:
        df_mobile = pd.read_excel(caminho_arquivo)
        df_mobile.columns = df_mobile.columns.str.strip()
        df_mobile = df_mobile.add_prefix('MOBILE_')
        
    elif 'Bipe_Produtos' in arquivo:
        df_bipe = pd.read_excel(caminho_arquivo)
        df_bipe.columns = df_bipe.columns.str.strip()
        df_bipe = df_bipe.add_prefix('BIPE_PROD_')
        
    elif 'Bipe_de_notas' in arquivo:
        df_bipe_notas = pd.read_excel(caminho_arquivo, sheet_name='Plan1')
        df_bipe_notas.columns = df_bipe_notas.columns.str.strip()
        df_bipe_notas = df_bipe_notas.add_prefix('BIPE_NOTAS_')

# Nota: O loop "for df in dfs: df.columns = ..." não é mais necessário 
# porque já fizemos o strip() individualmente acima.

In [53]:
df_esl

,ESL_DATA DO FRETE,ESL_REMETENTE/NOME FANTASIA,ESL_ENDEREÇO PRINCIPAL/LOGRADOURO,ESL_ENDEREÇO PRINCIPAL/NÚMERO,ESL_CIDADE/NOME,ESL_REMETENTE/DOCUMENTO,ESL_DESTINATÁRIO/NOME,ESL_ENDEREÇO PRINCIPAL/LOGRADOURO.1,ESL_ENDEREÇO PRINCIPAL/NÚMERO.1,ESL_ENDEREÇO PRINCIPAL/COMPLEMENTO,...,ESL_ÚLTIMA OCORRÊNCIA/OBSERVAÇÕES,ESL_MANIFESTO/NÚMERO.1,ESL_MANIFESTO/DATA.1,ESL_PESSOA/NOME.1,ESL_N° REFERÊNCIA,ESL_NOTA FISCAL/CHAVE NF-E,ESL_OBSERVAÇÕES DO CONTRIBUINTE/TEXTO,ESL_NOTA FISCAL/OBSERVAÇÕES,ESL_NOTA FISCAL/EMISSÃO,ESL_TABELA DE PREÇO/NOME
0,2025-12-01 00:05:00,CD640,JULIA GAIOLLI,740,Guarulhos,47960950089351,MARITZA ELENA LOZA SOTO,PROFESSOR GERALDO PASSARELLI,104,NaN,...,NaN,NaN,NaT,NaN,114518263,35251147960950089351550120001111031014691660,NaN,NaN,2025-11-30,MAGAZINE
1,2025-12-01 00:05:00,CD640,JULIA GAIOLLI,740,Guarulhos,47960950089351,JAILSON JOAO NOVAES,166,S/N,NaN,...,NaN,NaN,NaT,NaN,114518263,35251147960950089351550120001110171061042729,NaN,NaN,2025-11-30,MAGAZINE
2,2025-12-01 00:05:00,CD640,JULIA GAIOLLI,740,Guarulhos,47960950089351,ADRIANE DA SILVA OLIVEIRA,DOUTOR FRANCISCO TANCREDI,899,NaN,...,NaN,NaN,NaT,NaN,114518263,35251147960950089351550120001110781017587680,NaN,NaN,2025-11-30,MAGAZINE
3,2025-12-01 00:05:00,CD640,JULIA GAIOLLI,740,Guarulhos,47960950089351,CAMILA BOMFIM DA SILVA,ANDERSON DE MOURA,15,NaN,...,Recebido e.conferido conforme fotos,NaN,NaT,NaN,114518263,35251147960950089351550120001111001061923286,NaN,NaN,2025-11-30,MAGAZINE
4,2025-12-01 00:05:00,CD640,JULIA GAIOLLI,740,Guarulhos,47960950089351,DAMARIS PRISCILA DA SILVA,RUA ADOLFO D AVILA,06,NaN,...,NaN,NaN,NaT,NaN,114518263,35251147960950089351550120001111121076861282,NaN,NaN,2025-11-30,MAGAZINE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37635,2026-01-12 09:01:00,CD590,JULIA GAIOLLI,I,Guarulhos,47960950058804,ELIONAI DOS B COSTA,R TOMOICHI SHIMIZU,191,NaN,...,NaN,NaN,NaT,NaN,114790119,35260147960950058804550200007038711074761966,NaN,NaN,2026-01-10,MAGAZINE
37636,2026-01-12 09:01:00,CD590,JULIA GAIOLLI,I,Guarulhos,47960950058804,LUZIA APARECIDA DA SILVA NEVES,R OLIMPIA,193,NaN,...,NaN,NaN,NaT,NaN,114790119,35260147960950058804550200007038051089123590,NaN,NaN,2026-01-10,MAGAZINE
37637,2026-01-12 09:01:00,CD590,JULIA GAIOLLI,I,Guarulhos,47960950058804,FABIOLA DE OLIVEIRA THEODORO RIBEIRO,R NIGELA,130,NaN,...,NaN,NaN,NaT,NaN,114790119,35260147960950058804550200007038091027650340,NaN,NaN,2026-01-10,MAGAZINE
37638,2026-01-12 09:01:00,CD590,JULIA GAIOLLI,I,Guarulhos,47960950058804,ROSILENE DA SILVA,R NOVA SANTA ROSA,44,NaN,...,NaN,NaN,NaT,NaN,114790119,35260147960950058804550200007038191026498096,NaN,NaN,2026-01-10,MAGAZINE
